# Locality Sensitive hashing in Collaborative item-item filtering (inspired by week5 : SimilarItems)

Explain:
- LSH vs KNN, and why LSH is more efficient
- Present a scheme of how LSH algorithm operates
- Why we use Jaccard distance here (and how we make the decision for binary outcome, "likes":1/"dislikes":0 )

Imports

In [1]:
import numpy as np
import pandas as pd
import time
from datasketch import MinHash, MinHashLSHForest
import os
import hdf5_getters as hdf5_getters

In [2]:
################################ Alejandra's ############################################
path = 'MillionSongSubset'
songs_list = []

for (root, dirs, file) in os.walk(path):
    for f in file:
        songs_list.append(os.path.dirname(f))

            
print(len(songs_list))

## READ DATA PATH FROM FILE
songs_file_paths = []

for root, dirs, files in os.walk(os.path.abspath(path)):
    for file in files:        
        strs = os.path.join(root, file)
        new_strs = strs.replace('\\','/')
        songs_file_paths.append(new_strs)
        
### CREATE PANDAS TABLE

N = len(songs_file_paths)
data = []

for i in range(N):
    record = []
    
    # Open specific song path
    h5 = hdf5_getters.open_h5_file_read(songs_file_paths[i])
    
    #getters
    #artist_id = hdf5_getters.get_artist_id(h5)
    #artist_id = artist_id.decode("utf-8")     
    
    #artist_name  = hdf5_getters.get_artist_name(h5)
    #artist_name = artist_name.decode("utf-8") 
    
    #artist_location  = hdf5_getters.get_artist_location(h5)
    #artist_location = artist_location.decode("utf-8") 
    
    song_id = hdf5_getters.get_song_id(h5)
    song_id = song_id.decode("utf-8")
    
    song_name = hdf5_getters.get_title(h5)
    song_name = song_name.decode("utf-8")
    
    #song_hottness = hdf5_getters.get_song_hotttnesss(h5)
    
    #time_signature = hdf5_getters.get_time_signature(h5)
    
    artist_terms_ = hdf5_getters.get_artist_terms(h5)
    artist_terms = []
    for j in range(len(artist_terms_)):
        artist_terms.append(artist_terms_[j].decode("utf-8"))
    
    #artist_mbtags = hdf5_getters.get_artist_mbtags(h5)
    
    #mode = hdf5_getters.get_mode(h5)
    
    #year = hdf5_getters.get_year(h5)
    
    #latitude = hdf5_getters.get_artist_latitude(h5)
    #longitude = hdf5_getters.get_artist_longitude(h5)
    
    # Close file
    h5.close()
    
    #record.append(artist_id)
    #record.append(artist_name)
    #record.append(artist_location)
    record.append(song_id)
    record.append(song_name)
    #record.append(song_hottness)
    #record.append(time_signature)
    record.append(artist_terms)
    #record.append(artist_mbtags)
    #record.append(mode)
    #record.append(year)
    #record.append(latitude)
    #record.append(longitude)
    
    # Add
    data.append(record)
    
df = pd.DataFrame(data, columns=['song_id','song_name','artist_terms'])


10000


In [7]:
df.loc[df['song_id'] == 'SOOWVHQ12A8AE476A1']['song_name']

Series([], Name: song_name, dtype: object)

Read in the data:
    
    userID \tab songID \tab play_count

In [9]:
u_matrix = pd.read_csv('triplets_50000.txt', sep='\t', names = ['userID','songID', 'play_count'])

Pivot to tranform the data from long to wide:

In [10]:
u_matrix = u_matrix.pivot(index="userID", columns="songID", values="play_count")

In [11]:
u_matrix[356:370]

songID,SOAAAGQ12A8C1420C8,SOAACPJ12A81C21360,SOAACTC12AB0186A20,SOAADCB12A81C22AFA,SOAAEJI12AB0188AB5,SOAAEKX12A6D4F7E4E,SOAAFAC12A67ADF7EB,SOAAFYH12A8C13717A,SOAAIJG12AAA15D821,SOAAIWE12A8AE4706B,...,SOZZVFP12A8C140F14,SOZZVMW12AB0183B52,SOZZVNT12AF729EBC9,SOZZVWB12AB0189C30,SOZZWZV12A67AE140F,SOZZXAO12A58A7D379,SOZZYAO12A6701FF36,SOZZYDA12AB01824FB,SOZZYMH12AB0180A51,SOZZZFB12A8AE45CDC
userID,,,,,,,,,,,,,,,,,,,,,
5aa89364746d7f350d35a403da8f397bd9d32b41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5ab56ead71b71022f7043fef70a178b7035629b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5b27a99169fd85d31b9c46c6ebe16b42a40e9911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5b5bc0557423c6a58af47254385f75746fc5df90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5bbdf0419ed1b0f865890189ef1764152461a3fa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5c2510560078bc7d66ac6ca27d4c58aa11dab1d4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5c597664c9e728cd776d8d4b9bc9823ea15abf6e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5c855c96196f23de17179bf5ff3a46bbbe4ed45f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5d430e925662032ccc3245e88aca95ff6f49cd0d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# u_m = u_matrix.fillna(0) # Replace the NaN with 0s.

In [12]:
total_count_list = u_matrix.sum(axis=1, skipna=True)

In [7]:
print("The greatest listener's total play count is", np.max(total_count_list))
print("The lowest total play count of a user is", np.min(total_count_list))

The greatest listener's total play count is 2478.0
The lowest total play count of a user is 10.0


### Way num 1: normalize users and tranform to 0, 1

So it's a good idea, before we start, to normalize for "big" and "low" listeners:

In [13]:
u_normalized = u_matrix.sub(u_matrix.mean(axis=1, skipna=True), axis=0) # substract from each cell the row mean

In [14]:
u_normalized.shape

(1010, 27898)

In [15]:
u_normalized.head()

songID,SOAAAGQ12A8C1420C8,SOAACPJ12A81C21360,SOAACTC12AB0186A20,SOAADCB12A81C22AFA,SOAAEJI12AB0188AB5,SOAAEKX12A6D4F7E4E,SOAAFAC12A67ADF7EB,SOAAFYH12A8C13717A,SOAAIJG12AAA15D821,SOAAIWE12A8AE4706B,...,SOZZVFP12A8C140F14,SOZZVMW12AB0183B52,SOZZVNT12AF729EBC9,SOZZVWB12AB0189C30,SOZZWZV12A67AE140F,SOZZXAO12A58A7D379,SOZZYAO12A6701FF36,SOZZYDA12AB01824FB,SOZZYMH12AB0180A51,SOZZZFB12A8AE45CDC
userID,,,,,,,,,,,,,,,,,,,,,
0007c0e74728ca9ef0fe4eb7f75732e8026a278b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0039bd8483d578997718cdc0bf6c7c88b679f488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00498f4bab2bfeb17680113c7d9525ad5b0ad401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00a443baf550f4bbdd974ba73720abf2759166f3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0152fcbd02b172a874c75a57a913f0f0109ba272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Lets use the rule:

    if the norm play count is < 0 == the user didn't like the song
    if the norm play count is > 0 == the user likes the song

In [11]:
# u_binary = np.where(u_normalized[u_normalized.columns] < 0, 0, 1)

In [16]:
df_ = u_normalized.copy()

In [17]:
df_[df_ >= 0] = 1
df_[df_ < 0] = 0

In [18]:
df_.shape

(1010, 27898)

In [19]:
df_ = df_.T
df_

userID,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,03c90bfd09151973863c4cadd5a749cd7982abc0,...,fca15b7964a099b2860dfdb158a2430fb10c4384,fd13b9d49c54e00ff413fe3c095ba581c7fc611e,fd585aef5c32d3943bd6e7f9f39aa216ba659fd0,fdb815231ee1d66f383b80d279bd58769dfe59ff,fe76c9d535c5834e4a9b91c13e29be6460cb79c4,fe979a7b199de3ee8a78486c10e5ed13587fc359,fed37c4c49c9f217b3371c2f2c0e7541656e55cf,ff18ea9a13583f7f7aaa83719e0b22ce5618e9cf,ff4322e94814d3c7895d07e6f94139b092862611,ffadf9297a99945c0513cd87939d91d8b602936b
songID,,,,,,,,,,,,,,,,,,,,,
SOAAAGQ12A8C1420C8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAACPJ12A81C21360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAACTC12AB0186A20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAADCB12A81C22AFA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOAAEJI12AB0188AB5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOZZXAO12A58A7D379,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOZZYAO12A6701FF36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SOZZYDA12AB01824FB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Transform the data for the LSH algo
start_time = time.time()
cols = df_.columns.to_numpy() # the users 
vectors_list = [cols[x].tolist() for x in df_.eq(1).to_numpy()] # each vector is a song, contains users that liked that song
print('It took %s seconds.' %(time.time()-start_time))

It took 0.17598223686218262 seconds.


In [17]:
# sanity check
len(vectors_list) # should be equal to the 27898 columns (songs)

27898

In [21]:
df_new = pd.DataFrame(vectors_list, index = df_.index)
# df_new.shape # (27898, 49)


In [22]:
# drop rows with none
df_new_reduced = df_new.mask(df_new.eq('None')).dropna(how = 'all') 

In [23]:
df_new_reduced['users'] = df_new_reduced[df_new_reduced.columns].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df_new_reduced

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,users
songID,,,,,,,,,,,,,,,,,,,,,
SOAADCB12A81C22AFA,18dc95f354220b343e98ebbb7c8564291284ed9f,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,18dc95f354220b343e98ebbb7c8564291284ed9f
SOAAEKX12A6D4F7E4E,087b949b3fde63ddab83d61d4ba82a6f162bd50c,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,087b949b3fde63ddab83d61d4ba82a6f162bd50c
SOAAFYH12A8C13717A,5f07309f801582fc0fce1c176718d065fa6ea079,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,5f07309f801582fc0fce1c176718d065fa6ea079
SOAAKFY12A6D4F7B03,5d579eba844597fd498217472f07a83c26d71e57,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,5d579eba844597fd498217472f07a83c26d71e57
SOAAKPM12A58A77210,c3953e49b785c940f53a556abd9ca681c5ef48c5,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,c3953e49b785c940f53a556abd9ca681c5ef48c5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SOZZPSS12A6D4F3C71,5a905f000fc1ff3df7ca807d57edb608863db05d,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,5a905f000fc1ff3df7ca807d57edb608863db05d
SOZZTCU12AB0182C58,be0a4b64e9689c46e94b5a9a9c7910ee61aeb16f,ec0bb33707cdc61a6999b41585a0e1f39d0ae6d3,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"be0a4b64e9689c46e94b5a9a9c7910ee61aeb16f,ec0bb..."
SOZZTNF12A8C139916,76235885b32c4e8c82760c340dc54f9b608d7d7e,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,76235885b32c4e8c82760c340dc54f9b608d7d7e


In [24]:
data = df_new_reduced['users'].copy()

In [25]:
data = pd.DataFrame(data)

In [26]:
data.head()

,users
songID,
SOAADCB12A81C22AFA,18dc95f354220b343e98ebbb7c8564291284ed9f
SOAAEKX12A6D4F7E4E,087b949b3fde63ddab83d61d4ba82a6f162bd50c
SOAAFYH12A8C13717A,5f07309f801582fc0fce1c176718d065fa6ea079
SOAAKFY12A6D4F7B03,5d579eba844597fd498217472f07a83c26d71e57
SOAAKPM12A58A77210,c3953e49b785c940f53a556abd9ca681c5ef48c5


Choose parameters:

In [27]:
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

Create MiniHash forest

In [28]:
#Preprocess will split a string of text into individual tokens/shingles based on ",".
def preprocess(text):
    tokens = text.split()
    return tokens


In [29]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for users in data['users']:
        tokens = preprocess(users) # list of users
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest


In [30]:
forest = get_forest(data, permutations)

It took 19.525352478027344 seconds to build forest.


In [31]:
def predict(song_profile, database, perms, num_results, forest): # song_profile in list form
    start_time = time.time()
    
    m = MinHash(num_perm=perms)
    for users in song_profile:
        m.update(users.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['users']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result


In [32]:
num_recommendations = 20
song_profile = [ '5d5e0142e54c3bb7b69f548c2ee55066c90700eb'] # i made this random profile of an imaginary user manually, make it work with songID
result = predict(song_profile, data, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)


It took 0.007999181747436523 seconds to query forest.

 Top Recommendation(s) is(are) 
 songID
SOEKNHF12A3F1E9B8E    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOVHTGX12A8C143613    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOUNYVP12A6D4FB0BF    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOLXZIM12AB0186EEE    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOIOGFG12A8C13CD4F    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOVFSWY12A6D4FB0BE    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOOWVHQ12A8AE476A1    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOTUXYM12AF729E447    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
SOORYAT12A6D4FB0BD    5d5e0142e54c3bb7b69f548c2ee55066c90700eb
Name: users, dtype: object


In [39]:
df

,song_id,song_name,artist_terms
0,SOMZWCG12A8C13C480,I Didn't Mean To,"[hip hop, underground rap, g funk, alternative..."
1,SOCIWDW12A8C13D406,Soul Deep,"[blue-eyed soul, pop rock, blues-rock, beach m..."
2,SOXVLOJ12AB0189215,Amor De Cabaret,"[salsa, cumbia, tejano, ranchera, latin pop, l..."
3,SONHOTT12A8C13493C,Something Girls,"[pop rock, new wave, dance rock, rock, new rom..."
4,SOFSOCN12A8C143F5D,Face the Ashes,"[pop punk, ska punk, breakcore, alternative me..."
...,...,...,...
9995,SOLXXPY12A67ADABA0,The Hanged Man,"[sympho black metal, black metal, heavy metal,..."
9996,SOAYONI12A6D4F85C8,The Wonderful World Of The Young,"[ballad, disco, northern soul, chanson, smooth..."
9997,SOJZLAJ12AB017E8A2,Sentimental Man,"[lovers rock, reggae, roots reggae, dub, orche..."
9998,SORZSCJ12A8C132446,Zydeco In D-Minor,"[zydeco, cajun]"


In [38]:

for song in result.index:
    print(df.loc[df['song_id'] == song]['song_name'])

Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)
Series([], Name: song_name, dtype: object)


## Way num. 2 pick for each song the top n users 

In [61]:
n = 10
t3 = u_matrix.T.apply(lambda x: pd.Series(x.nlargest(n).index), axis=1)

In [62]:
print(u_matrix.T.shape,'\n', t3.shape)

(27898, 1010) 
 (27898, 10)


In [63]:
t3.head(10)

,0,1,2,3,4,5,6,7,8,9
songID,,,,,,,,,,
SOAAAGQ12A8C1420C8,3ff7a31452eeabd7a4e07f0d243c674e3d0adf46,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAACPJ12A81C21360,fb2f2c0e39e233622c300c232a6287738007e34a,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAACTC12AB0186A20,cd7baa603618c40b9290bd05112061afefdde8ef,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAADCB12A81C22AFA,18dc95f354220b343e98ebbb7c8564291284ed9f,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAAEJI12AB0188AB5,28b232e7ecb32c47c05b795a017786d4be96ef7e,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAAEKX12A6D4F7E4E,087b949b3fde63ddab83d61d4ba82a6f162bd50c,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAAFAC12A67ADF7EB,76235885b32c4e8c82760c340dc54f9b608d7d7e,7e2b716907a9a295d92c2f35d7e260aeee2b32cf,c24ec42f0e449ff39a95a01f0795f833b898f71b,fdb815231ee1d66f383b80d279bd58769dfe59ff,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031
SOAAFYH12A8C13717A,5f07309f801582fc0fce1c176718d065fa6ea079,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2
SOAAIJG12AAA15D821,32fe8a8c93847b8771d357411d3aa73413d72779,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2


In [70]:
# Tranform in the prefered format for the function 
c = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
t3['users'] = t3.iloc[:, :].apply(",".join, axis=1)

In [71]:
t3.head()

,0,1,2,3,4,5,6,7,8,9,users
songID,,,,,,,,,,,
SOAAAGQ12A8C1420C8,3ff7a31452eeabd7a4e07f0d243c674e3d0adf46,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,"3ff7a31452eeabd7a4e07f0d243c674e3d0adf46,0007c..."
SOAACPJ12A81C21360,fb2f2c0e39e233622c300c232a6287738007e34a,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,"fb2f2c0e39e233622c300c232a6287738007e34a,0007c..."
SOAACTC12AB0186A20,cd7baa603618c40b9290bd05112061afefdde8ef,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,"cd7baa603618c40b9290bd05112061afefdde8ef,0007c..."
SOAADCB12A81C22AFA,18dc95f354220b343e98ebbb7c8564291284ed9f,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,"18dc95f354220b343e98ebbb7c8564291284ed9f,0007c..."
SOAAEJI12AB0188AB5,28b232e7ecb32c47c05b795a017786d4be96ef7e,0007c0e74728ca9ef0fe4eb7f75732e8026a278b,0039bd8483d578997718cdc0bf6c7c88b679f488,00498f4bab2bfeb17680113c7d9525ad5b0ad401,00a443baf550f4bbdd974ba73720abf2759166f3,0152fcbd02b172a874c75a57a913f0f0109ba272,01655ae6bc52e29c9cd100a7dde4e9eeae5e4031,019d0d1c7a01f8736ba59a124160e5fc70666db7,02c2dbc1119bca9513259f1b1903432538037d10,037167e01a2b265b8ee59694db943f9556876be2,"28b232e7ecb32c47c05b795a017786d4be96ef7e,0007c..."


In [72]:
data2 = t3.drop(t3.columns[range(0,10)], axis=1)

In [73]:
data2

,users
songID,
SOAAAGQ12A8C1420C8,"3ff7a31452eeabd7a4e07f0d243c674e3d0adf46,0007c..."
SOAACPJ12A81C21360,"fb2f2c0e39e233622c300c232a6287738007e34a,0007c..."
SOAACTC12AB0186A20,"cd7baa603618c40b9290bd05112061afefdde8ef,0007c..."
SOAADCB12A81C22AFA,"18dc95f354220b343e98ebbb7c8564291284ed9f,0007c..."
SOAAEJI12AB0188AB5,"28b232e7ecb32c47c05b795a017786d4be96ef7e,0007c..."
...,...
SOZZXAO12A58A7D379,"4208d4ac45e7caab7167a4ea6d34e759a6b9a1fc,0007c..."
SOZZYAO12A6701FF36,"5a905f000fc1ff3df7ca807d57edb608863db05d,0007c..."
SOZZYDA12AB01824FB,"2c218a60b3d777e9e12d56c2e065a9644b5e5f41,0007c..."


Now we are ready to use the LSH function

In [79]:
forest2 = get_forest(data2, permutations)

It took 58.46970200538635 seconds to build forest.


In [33]:
num_recommendations = 10
user_profile = [ '00498f4bab2bfeb17680113c7d9525ad5b0ad401'] # i made this random profile of an imaginary user manually, make it work with songID
result2 = predict(song_profile, data2, permutations, num_recommendations, forest2)
print('\n Top Recommendation(s) is(are) \n', result2)

NameError: name 'data2' is not defined